# Тестовое задание в компанию Адвентум
подготовил: [Вова Сизов](mailto:vladimirsizov@yandex.ru)

<hr>

Тестовое задание:
Возьмите свою страницу Вконтакте, соберите по ней таблицу с датой постов и количеством лайков и напишите SQL-запросы, которые позволят ответить на вопрос: что больше всего влияет на количество лайков: время суток публикации, день недели или промежуток между постами.

В качестве ответа приложите таблицу в формате CSV, SQL-запросы с необходимыми пояснениями и выводом. Вместо своей страницы можно взять станицу друга, в любом случае укажите адрес страницы.

<hr>

Для начала соберём данные из vk.com  
Так как у меня не достаточно лайков для сравнения я вопользовался страницей друга [András Arató](https://vk.com/painharold) "Гарольда скрывающего боль"

In [1]:
import pandas as pd
import datetime
import vk_api

In [2]:
# token
access_token='vk1.a.ikaciQ-LymzbiSsBOpJKkQ7SuBNb5O4rnZdk29bJ6Ow9tsZHw-iW_SlKWoITr7z6GCiHeE_Oqw5Aej9r5RfEqxXoq8ig'

In [3]:
# создаём подключение
vk_session = vk_api.VkApi(token=access_token)
vk = vk_session.get_api()

In [4]:
# https://dev.vk.com/ru/method/wall.get
result = vk.wall.get(domain='painharold', count=100)

Получаем датафрейм с датами публикаций на стене профиля и количеством лайков под постами

In [5]:
data = []
for publish in result['items']:
    date = datetime.datetime.utcfromtimestamp(publish['date'])
    likes = publish['likes']['count']
    data.append([date, likes])

data_result = pd.DataFrame(data=data, columns=['date', 'likes'])
data_result

,date,likes
0,2023-07-19 14:23:43,665
1,2023-05-11 10:59:55,635
2,2023-05-05 21:33:38,0
3,2023-04-23 07:21:35,610
4,2023-04-10 12:54:26,133
...,...,...
95,2021-02-24 15:35:00,93
96,2021-02-23 16:55:00,205
97,2021-02-22 13:45:00,389
98,2021-02-18 13:35:23,0


In [6]:
# запишем результат в файл CSV
data_result.to_csv('vladimir_sizov_test.csv', index=False)

<hr>

Мы воспользуемся сервисом https://sqliteonline.com/ где мы можем создать таблицы в PostgreSQL и cформировать к ним SQL запросы.   
Создадим таблицу:

In [ ]:
CREATE TABLE IF NOT EXISTS posts (
    id SERIAL PRIMARY KEY, 
    date TIMESTAMP,
    likes INTEGER
    );

Наполним таблицу этими данными:

In [8]:
data = []
print('INSERT INTO posts(date, likes) VALUES')
for publish in range(len(result['items'])):
    date = datetime.datetime.utcfromtimestamp(result['items'][publish]['date']).strftime('%Y-%m-%d %H:%M:%S')
    likes = result['items'][publish]['likes']['count']
    if publish < len(result['items']) - 1:
        print((date, likes),',')
    else:
        print((date, likes),';')

INSERT INTO posts(date, likes) VALUES
('2023-07-19 14:23:43', 665) ,
('2023-05-11 10:59:55', 635) ,
('2023-05-05 21:33:38', 0) ,
('2023-04-23 07:21:35', 610) ,
('2023-04-10 12:54:26', 133) ,
('2023-04-04 12:49:19', 690) ,
('2023-04-02 20:56:48', 402) ,
('2023-02-15 20:44:44', 1060) ,
('2023-01-24 20:16:42', 678) ,
('2023-01-10 20:44:10', 0) ,
('2022-12-31 21:01:53', 1025) ,
('2022-12-05 21:50:26', 876) ,
('2022-11-30 19:22:45', 421) ,
('2022-11-20 22:18:07', 1139) ,
('2022-11-08 17:17:11', 297) ,
('2022-10-27 21:37:15', 1116) ,
('2022-10-11 09:07:39', 0) ,
('2022-09-19 21:35:58', 386) ,
('2022-09-19 21:22:52', 1467) ,
('2022-09-12 10:25:12', 707) ,
('2022-08-08 11:04:50', 952) ,
('2022-06-28 10:37:44', 0) ,
('2022-06-23 20:56:42', 1100) ,
('2022-06-19 09:54:20', 0) ,
('2022-06-04 22:03:33', 1) ,
('2022-05-30 17:02:49', 1083) ,
('2022-04-25 11:11:47', 362) ,
('2022-04-12 03:43:54', 6) ,
('2022-03-31 10:32:11', 125) ,
('2022-03-30 21:29:31', 1513) ,
('2022-03-25 20:25:44', 3) ,
('2022-03

<hr>

Теперь нам важно определиться, что позволят ответить на вопрос: что больше всего влияет на количество лайков: время суток публикации, день недели или промежуток между постами?

Мы будем сравнивать стандартное отклонение количества лайков среднего значения при определённой группировке (по дням недели, времени суток, дней между публикациями).  
Наибольшее значение стандартного отклонения покажет максимальное различие в количестве лайков его мы будем принимать как влияние.

In [ ]:
-- разница по времени суток(часы) показала значение 322

SELECT
    STDDEV(avg_likes)
FROM (
    SELECT
        AVG(likes) AS avg_likes,
        DATE_PART('hour', date) AS hour
    FROM posts
    GROUP BY hour
    ) AS h

In [ ]:
-- разница по дням недели показала значение 123

SELECT
    STDDEV(avg_likes)
FROM (
    SELECT
        AVG(likes) AS avg_likes,
        DATE_PART('isodow', date) AS day_week
    FROM posts
    GROUP BY day_week
    ) AS w

In [ ]:
-- разница в количестве дней(десять дней шаг) между публикациями 183
SELECT
    STDDEV(avg_likes)
FROM (
    SELECT 
        AVG(likes) AS avg_likes,
        day_diff
    FROM (
        SELECT
            likes,
            ((DATE_PART('day', date - LAG(date, 1) OVER(ORDER BY date)))/10)::INT AS day_diff
        FROM posts
    ) as dd
    WHERE day_diff IS NOT NULL
    GROUP BY day_diff
    ) AS d

In [ ]:
-- лучшее время публикации 11

SELECT
    *
FROM (
    SELECT
        AVG(likes) AS avg_likes,
        DATE_PART('hour', date) AS hour
    FROM posts
    GROUP BY hour
    ) AS h
ORDER BY avg_likes DESC
LIMIT 1

# Вывод:
Наибольшее влияние на количество лайков оказывает время суток публикации, лучшее время 11 часов утра, второе по значимости 19.